In [0]:
from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, AveragePooling2D, Dense, Activation, Flatten, Reshape, Add, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import numpy as np

def relu6(inputs):
    '''
        Performs the ReLU6 activation function for the bottleneck stage of the MobileNet V2
        Inputs:
            inputs: the layer with the inputs for the activation function
        Return:
            Min value between the value of the regular ReLU function and 6
    '''
    
    return K.relu(inputs,max_value=6)

def bottleneck(inputs, t, alpha, num_filters, kernel_sz=(3,3),stride=(1,1),pad='same',residual=False,dropout=False,dropout_perc=0.1):    
    '''
        Performs the bottleneck stage of the MobileNet V2
        Inputs:
            inputs: the layer with the inputs
            t: the value used to increase the number of filters of the expansion stage
            alpha: width multiplier that controls the number of filters of the output tensor
            num_filters: number of filters of the output tensor
            kernel_sz = kernel size of the filter
            stride: stride of the kernel
            pad: padding of the filter
            residual: parameter that determine the sum of the input and output of the bottleneck stage
            dropout: determine if dropout will be performed 
            dropout_perc: percentage of neurons that will be set to zero
        Return:
            x: the result of the bottleneck stage
    '''    
    
    # Get the index of the input 4D tensor that represents the number of channels of the image
    # -1 can also represent the last element of the tensor
    channel_idx = 1 if K.image_data_format == 'channels_first' else -1
    
    # Number of filters for the expansion convolution
    num_filters_exp = K.int_shape(inputs)[channel_idx] * t    
    
    # Number of filters of the projection convolution
    num_filters_proj = int(num_filters * alpha)
    
    # Expansion layer
    x = Conv2D(filters=num_filters_exp,kernel_size=(1,1),strides=(1,1),padding=pad)(inputs)
    x = BatchNormalization()(x)
    x = Activation(relu6)(x)
    
    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=kernel_sz,strides=stride,depth_multiplier=1,padding=pad)(x)
    x = BatchNormalization()(x)
    x = Activation(relu6)(x)
    
    # Projection convolution
    x = Conv2D(filters=num_filters_proj,kernel_size=(1,1),strides=(1,1),padding=pad)(x)
    x = BatchNormalization()(x)
    
    if (residual == True):
        x = Add()([x,inputs])
        
    if (dropout == True):
      x = Dropout(dropout_perc)(x)
    
    return x

def depthwise_block(inputs,stride,kernel_sz=(3,3),pad='same'):
    '''
        Function that performs the depthwise convolution
        Inputs:
            inputs:    the input shape of the depthwise convolution
            kernel_sz: a tuple that indicates the size of the filtering kernel
            stride:    a tuple that indicates the strides of the kernel
        Return:
            x: the result of the depthwise convolution
    '''
        
    x = DepthwiseConv2D(kernel_size=kernel_sz,strides=stride,depth_multiplier=1,padding=pad)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
        
    return x

def pointwise_block(inputs,num_filters,alpha,kernel_sz=(1,1),stride=(1,1),pad='same',dropout=False,dropout_perc=0.1):
    '''
        Function that performs the pointwise convolution
        Inputs:
            inputs:      the input shape of the depthwise convolution
            num_filters: number of filters to be used in the convolution
            kernel_sz:   a tuple that indicates the size of the filtering kernel
            stride:      a tuple that indicates the strides of the kernel
            dropout: determine if dropout will be performed 
            dropout_perc: percentage of neurons that will be set to zero            
        Return:
            x: the result of the pointwise convolution
    '''    
    
    # Number of filters based on width multiplier reported in the original paper
    n_fil = int(num_filters * alpha)    
    
    x = Conv2D(filters=n_fil,kernel_size=kernel_sz,padding=pad)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    if (dropout == True):
      x = Dropout(dropout_perc)(x)
    
    return x



def MobileNetV1(input_shape,num_units,filters=32,kernel_sz=(3,3),stride=(2,2),alp=1,ro=1,dropout_perc=0.1):
    input_shape = (int(input_shape[0] * ro), int(input_shape[1] * ro), input_shape[2])
    
    inputs = Input(shape=input_shape)
    
    # Regular convolution
    x = Conv2D(filters=filters,kernel_size=kernel_sz,strides=stride)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(dropout_perc)(x)
    
    # First depthwise-pointwise block
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=64,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    # Second depthwise-pointwise block
    x = depthwise_block(x,kernel_sz=(3,3),stride=(2,2))
    x = pointwise_block(x,num_filters=128,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    # Third depthwise-pointwise block
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=128,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    # Fourth depthwise-pointwise block
    x = depthwise_block(x,kernel_sz=(3,3),stride=(2,2))
    x = pointwise_block(x,num_filters=256,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)    
    
    # Fifth depthwise-pointwise block
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=256,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)

    # Sixth depthwise-pointwise block
    x = depthwise_block(x,kernel_sz=(3,3),stride=(2,2))
    x = pointwise_block(x,num_filters=512,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    # Seventh depthwise-pointwise block (repeated five times)
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=512,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=512,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=512,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)

    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=512,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=512,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc) 
    
    # Eight depthwise-pointwise block
    x = depthwise_block(x,kernel_sz=(3,3),stride=(2,2))
    x = pointwise_block(x,num_filters=1024,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)

    # Nineth depthwise-pointwise block    
    x = depthwise_block(x,kernel_sz=(3,3),stride=(1,1))
    x = pointwise_block(x,num_filters=1024,alpha=alp,stride=(1,1),dropout=True,dropout_perc=dropout_perc)    
    
    # Pooling layer
    # Pooling size correction due to the resolution multiplier parameter
    pool_size = int(np.round(7*ro))
    x = AveragePooling2D(padding='valid',pool_size=(pool_size,pool_size),strides=(1,1))(x)    
    
    x = Flatten()(x)
    
    # Fully connected layer
    x = Dense(units=1024,activation='relu')(x)
    
    # Softmax layer
    output = Dense(num_units,activation='softmax')(x)
    
    model = Model(inputs,output)
    
    return model

def MobileNetV2(input_shape, num_units, filters=32, kernel_sz=(3,3),stride=(2,2),alp=1,ro=1,dropout_perc=0.1):
    input_shape = (int(input_shape[0] * ro), int(input_shape[1] * ro), input_shape[2])
    
    inputs = Input(shape=input_shape)
    
    # Regular convolution
    x = Conv2D(filters=filters,kernel_size=kernel_sz,strides=stride)(inputs)
    x = BatchNormalization()(x)
    x = Activation(relu6)(x)
    x = Dropout(dropout_perc)(x)

    # First bottleneck convolution
    x = bottleneck(x,t=1,alpha=alp,num_filters=16,kernel_sz=(3,3),stride=(1,1),dropout=True,dropout_perc=dropout_perc)

    # Second bottleneck convolution (peformed 2 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=24,kernel_sz=(3,3),stride=(2,2),dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=24,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Third bottleneck convolution (peformed 3 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=32,kernel_sz=(3,3),stride=(2,2))
    x = bottleneck(x,t=6,alpha=alp,num_filters=32,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=32,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Fourth bottleneck convolution (performed 4 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(2,2))
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=64,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Fifth bottleneck convolution (performed 3 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=96,kernel_sz=(3,3),stride=(1,1))
    x = bottleneck(x,t=6,alpha=alp,num_filters=96,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=96,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Sixth bottleneck convolution (performed 3 times)
    x = bottleneck(x,t=6,alpha=alp,num_filters=160,kernel_sz=(3,3),stride=(2,2))
    x = bottleneck(x,t=6,alpha=alp,num_filters=160,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    x = bottleneck(x,t=6,alpha=alp,num_filters=160,kernel_sz=(3,3),stride=(1,1), residual=True,dropout=True,dropout_perc=dropout_perc)
    
    # Seventh bottleneck convolution (performed 1 time)
    x = bottleneck(x,t=6,alpha=alp,num_filters=320,kernel_sz=(3,3),stride=(1,1),dropout=True,dropout_perc=dropout_perc)
    
    # Eigth layer (regular convolution)
    x = Conv2D(filters=1280, kernel_size=(1,1), strides=(1,1), padding='same')(x)
    
    # Pooling layer
    # Pooling size correction due to the resolution multiplier parameter
    pool_size = int(np.round(7*ro))
    x = AveragePooling2D(padding='valid',pool_size=(pool_size,pool_size),strides=(1,1))(x)    
    
    x = Conv2D(filters=num_units,kernel_size=(1,1),strides=(1,1), padding='same')(x)
    
    output = Reshape((num_units,))(Activation(activation='softmax')(x))
    
    return Model(inputs,output)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
alpha = 1
ro = 1

img_size = 224

target_sz = int(img_size * ro)

batch_sz = 32

epo = 200

image_generator = ImageDataGenerator(rotation_range=15,
                                      rescale=1./255,
                                      shear_range=0.2,
                                      zoom_range=0.1,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
train_generator = image_generator.flow_from_directory('/content/gdrive/My Drive/Imagens/Airplane models/Treinamento',
                                                     target_size=(target_sz,target_sz),
                                                     color_mode='rgb',
                                                     batch_size=batch_sz,
                                                     class_mode='categorical',
                                                     shuffle=True)

image_generator = ImageDataGenerator(rescale=1./255)
test_generator = image_generator.flow_from_directory('/content/gdrive/My Drive/Imagens/Airplane models/Teste',
                                                     target_size=(target_sz,target_sz),
                                                     color_mode='rgb',
                                                     batch_size=batch_sz,
                                                     class_mode='categorical',
                                                     shuffle=True)

In [0]:
model_v1 = MobileNetV1((img_size,img_size,3),num_units=2,alp=alpha,ro=ro,dropout_perc=0.2)

optimizer = Adam(lr=0.001)

model_v1.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

model_check_point_v1 = ModelCheckpoint(filepath='/content/gdrive/My Drive/Colab Notebooks/Experimento 2/model_mb_v1.hdf5',monitor='val_loss',verbose=1,save_best_only=False,save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.7,patience=10,verbose=1,min_lr=0.0001)

step_size_train = train_generator.n/train_generator.batch_size
step_size_test = test_generator.n/test_generator.batch_size

history_train_v1 = model_v1.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=epo, validation_data=test_generator,validation_steps=step_size_test,
                    workers=8,
                    use_multiprocessing=False,
                    callbacks=[reduce_lr])

model_v1.save('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/model_mb_v1_final.hdf5')

In [0]:
import pandas as pd
hist_df_v1 = pd.DataFrame(history_train_v1.history)

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/history_mobilenet_v1.json',mode='w') as f:
  hist_df_v1.to_json(f)

In [0]:
from keras.models import load_model
mb_v1 = load_model('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/model_mb_v1_final.hdf5')

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
plt.plot(history_train_v1.history['loss'],c='r',label='Training loss')
plt.plot(history_train_v1.history['val_loss'],c='b',label='Validation loss')
plt.title('Loss MobileNet V1')
plt.legend()
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/loss_mbnet_v1.jpg')

In [0]:
plt.plot(history_train_v1.history['acc'],c='r',label='Training accuracy')
plt.plot(history_train_v1.history['val_acc'],c='b',label='Validation accuracy')
plt.title('Accuracy MobileNet V1')
plt.legend()
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/accuracy_mbnet_v1.jpg')

In [0]:
image_generator = ImageDataGenerator(rescale=1./255)
test_generator = image_generator.flow_from_directory('/content/gdrive/My Drive/Imagens/Airplane models/Teste',
                                                     target_size=(target_sz,target_sz),
                                                     color_mode='rgb',
                                                     batch_size=batch_sz,
                                                     class_mode='categorical',
                                                     shuffle=False)

Found 822 images belonging to 2 classes.


In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = mb_v1.predict_generator(test_generator,workers=8,steps=step_size_test)
y_pred = np.argmax(y_pred,axis=1)
y_true = test_generator.classes
target_names = ['A320','B737']
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred,target_names=target_names))

In [0]:
from keras.preprocessing import image
img = image.load_img('/content/gdrive/My Drive/Imagens/Airplane models/Validação/B737/boeing_737_700_566386.jpg',target_size=(target_sz,target_sz))
x = image.img_to_array(img)/255.

x = x.reshape((-1,target_sz,target_sz,3))

pred = mb_v1.predict(x)

predicted_class_index = np.argmax(pred,axis=1)

labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
prediction = [labels[k] for k in predicted_class_index]

plt.figure()
plt.imshow(img)
plt.title('Airplane model: '+str(prediction))
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/Experimento 2/B737_4.jpg')
pred